## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/115-googleq-a-train-bert-base-uncased-quest-ans/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/115-googleq-a-train-bert-base-uncased-quest-ans/model_0.h5', '/kaggle/input/115-googleq-a-train-bert-base-uncased-quest-ans/model_1.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

question_features = ['question_title', 'question_body']
answer_features = ['question_title', 'answer']

# Test features
X_test_question = compute_input_arays_2(test, question_features, tokenizer, MAX_SEQUENCE_LENGTH, ignore_first_sep=False)
X_test_answer = compute_input_arays_2(test, answer_features, tokenizer, MAX_SEQUENCE_LENGTH, ignore_first_sep=False)

# Model

In [7]:
def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x = Concatenate()([x_question, x_answer])
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test_question + X_test_answer) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.905410,0.571597,0.394633,0.492526,0.466339,0.532449,0.666888,0.631341,0.521299,...,0.929321,0.874201,0.546288,0.957009,0.928817,0.760681,0.049011,0.046678,0.806457,0.924850
1,46,0.883866,0.537816,0.009013,0.651762,0.775045,0.879678,0.545810,0.499453,0.115653,...,0.748040,0.952236,0.676108,0.963503,0.977205,0.873759,0.893412,0.109493,0.110086,0.890273
2,70,0.897191,0.614895,0.073448,0.641294,0.810875,0.800822,0.640026,0.491132,0.187969,...,0.900155,0.893166,0.576753,0.953262,0.954956,0.848092,0.113030,0.061332,0.749074,0.925397
3,132,0.843042,0.500711,0.015113,0.672521,0.738904,0.879967,0.539435,0.399157,0.162370,...,0.766022,0.951825,0.732118,0.966221,0.978791,0.901975,0.775962,0.160852,0.627597,0.882094
4,200,0.908539,0.380506,0.058087,0.751122,0.747521,0.807398,0.647912,0.605649,0.127805,...,0.750300,0.928531,0.649110,0.952239,0.966979,0.847067,0.215953,0.094923,0.675308,0.924115


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.891217,0.565504,0.058445,0.645355,0.770800,0.794858,0.597592,0.506333,0.244087,...,0.811154,0.929826,0.662578,0.959372,0.965063,0.864195,0.467262,0.130493,0.566772,0.910133
std,2812.670060,0.039005,0.114489,0.086260,0.104361,0.096220,0.129891,0.046376,0.083241,0.168691,...,0.075102,0.024224,0.049867,0.010571,0.011870,0.037409,0.303167,0.048182,0.246239,0.020096
min,39.000000,0.762963,0.323593,0.004553,0.258809,0.287605,0.191865,0.504937,0.355170,0.022556,...,0.629987,0.844916,0.515365,0.916258,0.919235,0.701766,0.007288,0.024054,0.068805,0.845766
25%,2572.000000,0.862805,0.476717,0.012869,0.585679,0.730504,0.757405,0.561489,0.443461,0.108217,...,0.746737,0.916477,0.630591,0.953226,0.958373,0.840612,0.151722,0.096060,0.362941,0.895134
50%,5093.000000,0.894537,0.550380,0.021016,0.654152,0.780424,0.845904,0.592686,0.484608,0.199538,...,0.807802,0.935072,0.664382,0.961970,0.966712,0.871793,0.508021,0.134996,0.576985,0.911284
75%,7482.000000,0.921398,0.654894,0.058290,0.720611,0.831024,0.880406,0.628592,0.552843,0.345529,...,0.878872,0.947855,0.696558,0.966561,0.974519,0.892207,0.754596,0.161544,0.764847,0.923890
max,9640.000000,0.972069,0.851914,0.617385,0.896645,0.944191,0.939158,0.746751,0.757404,0.786659,...,0.943614,0.968038,0.798887,0.977503,0.984668,0.940146,0.937721,0.266381,0.981645,0.963967
